In [21]:
import pandas as pd
import numpy as np
import os

gff3Cols=["seqid","source","type","start","end","score","strand","phase","attributes"]
outDir="./res_tmp_full"

In [22]:
df=pd.read_csv(outDir+"/dfGFF_gmap.gtf",sep="\t",names=gff3Cols)

df["id"]=df.attributes.str.split("ID=",expand=True)[1].str.split(";",expand=True)[0]
dfE=df[df["type"]=="exon"].reset_index(drop=True)
dfE["transID"]=dfE.attributes.str.split("Parent=",expand=True)[1].str.split(";",expand=True)[0]
transcriptSet=set(dfE["transID"])
dfT=df[df["id"].isin(transcriptSet)].reset_index(drop=True)
assert len(set(dfT["id"]))==len(transcriptSet)
dfT["transID"]=dfT["id"]

# here's the time to deal with inconsistencies
# first remove any enties that do not have a parent - those do not conform to the expected structure that we define
# however, this way we remove PSEUDOGENES, which may not be the desired effect - need to think more/discuss
dfE=dfE[dfE["transID"].isin(dfT["transID"])].reset_index(drop=True)
dfT["type"]="transcript"

dfE=dfE.merge(dfT[["transID"]],how="left",left_on="transID",right_on="transID")

# now need to concatenate all these three dataframes
dfM=pd.concat([dfT[["seqid","source","type","start","end","score","strand","phase","attributes","transID"]],dfE[["seqid","source","type","start","end","score","strand","phase","attributes","transID"]]],axis=0).reset_index(drop=True)
dfM["type"]=pd.Categorical(dfM["type"],categories=["transcript","exon"],ordered=True)
dfM=dfM.sort_values(by=["transID","type"]).reset_index(drop=True)
del dfE
del dfT
del df

dfM["start"]=dfM["start"].astype(int)
dfM["end"]=dfM["end"].astype(int)-1
# some pseudogenes are still present

dfM_g=dfM[dfM["type"].isin(["exon"])][["seqid","strand","type","start","end","transID"]].reset_index(drop=True)
dfM_g=dfM_g.sort_values(by=["transID","start"]).reset_index(drop=True)
dfM_g["ns"]=dfM_g.start.astype(str).shift(-1)
dfM_g["nid"]=dfM_g.transID.shift(-1)
dfM_g["ns"]=np.where(dfM_g["transID"]==dfM_g["nid"],dfM_g["ns"],"***")
dfM_g["ns"]=dfM_g.ns.astype(str)+"+"
dfM_g["ends"]=dfM_g.end.astype(str)+"+"
dfM_g=dfM_g[~(dfM_g['ns']=="***+")].reset_index(drop=True)
dfM_g=dfM_g.groupby(by="transID").agg({'start':'min',\
                                       'ns':'sum',\
                                       'ends':'sum',\
                                       'end':'max',\
                                       'strand':'min',\
                                       'seqid':'min',\
                                       'nid':'count'})
dfM_g.rename({"nid":'count'},axis=1,inplace=True)
dfM_g.reset_index(inplace=True)
dfM_g.columns=['transID','start_min',"ns",'end','end_max','strand','seqid','count']
dfM_g["ns"]=dfM_g["ns"].str.rstrip("***+")
dfM_g["end"]=dfM_g["end"].str.rstrip("***+")
dfM_g["chain"]=dfM_g['seqid']+":"+dfM_g['strand']+'@'+dfM_g['end']+'-'+dfM_g['ns']
dfM_g=dfM_g[['transID','start_min','end_max','chain','count']]

del transcriptSet
dfM=dfM[gff3Cols+["transID"]]
dfM

,seqid,source,type,start,end,score,strand,phase,attributes,transID
0,CM000670.2,ref,transcript,128564229,128564679,.,-,.,ID=0,0
1,CM000670.2,ref,exon,128564229,128564679,.,-,.,Parent=0,0
2,CM000670.2,ref,transcript,128559028,128566526,.,-,.,ID=1,1
3,CM000670.2,ref,exon,128559028,128559576,.,-,.,Parent=1,1
4,CM000670.2,ref,exon,128560240,128560548,.,-,.,Parent=1,1
5,CM000670.2,ref,exon,128561733,128561789,.,-,.,Parent=1,1
6,CM000670.2,ref,exon,128566076,128566526,.,-,.,Parent=1,1
7,CM000670.2,ref,transcript,12378651,12380285,.,-,.,ID=10,10
8,CM000670.2,ref,exon,12378651,12380285,.,-,.,Parent=10,10
9,CM000670.2,ref,transcript,143790920,144172229,.,-,.,ID=100,100


In [24]:
df=pd.read_csv(outDir+"/chessPrim.gff",sep='\t',names=gff3Cols)
df=df[df["type"].isin(['transcript','exon'])].reset_index(drop=True)

df["id"]=df.attributes.str.split("ID=",expand=True)[1].str.split(";",expand=True)[0]
dfE=df[df["type"]=="exon"].reset_index(drop=True)
dfE["transID"]=dfE.attributes.str.split("Parent=",expand=True)[1].str.split(";",expand=True)[0]
transcriptSet=set(dfE["transID"])
dfT=df[df["id"].isin(transcriptSet)].reset_index(drop=True)
assert len(set(dfT["id"]))==len(transcriptSet)
dfT["transID"]=dfT["id"]

# here's the time to deal with inconsistencies
# first remove any enties that do not have a parent - those do not conform to the expected structure that we define
# however, this way we remove PSEUDOGENES, which may not be the desired effect - need to think more/discuss
dfE=dfE[dfE["transID"].isin(dfT["transID"])].reset_index(drop=True)
dfT["type"]="transcript"

dfE=dfE.merge(dfT[["transID"]],how="left",left_on="transID",right_on="transID")

# now need to concatenate all these three dataframes
dfM_2=pd.concat([dfT[["seqid","source","type","start","end","score","strand","phase","attributes","transID"]],dfE[["seqid","source","type","start","end","score","strand","phase","attributes","transID"]]],axis=0).reset_index(drop=True)
dfM_2["type"]=pd.Categorical(dfM_2["type"],categories=["transcript","exon"],ordered=True)
dfM_2=dfM_2.sort_values(by=["transID","type"]).reset_index(drop=True)
del dfE
del dfT
del df

dfM_2["start"]=dfM_2["start"].astype(int)
dfM_2["end"]=dfM_2["end"].astype(int)
# some pseudogenes are still present

dfM_2_g=dfM_2[dfM_2["type"].isin(["exon"])][["seqid","strand","type","start","end","transID"]].reset_index(drop=True)
dfM_2_g=dfM_2_g.sort_values(by=["transID","start"]).reset_index(drop=True)
dfM_2_g["ns"]=dfM_2_g.start.astype(str).shift(-1)
dfM_2_g["nid"]=dfM_2_g.transID.shift(-1)
dfM_2_g["ns"]=np.where(dfM_2_g["transID"]==dfM_2_g["nid"],dfM_2_g["ns"],"***")
dfM_2_g["ns"]=dfM_2_g.ns.astype(str)+"+"
dfM_2_g["ends"]=dfM_2_g.end.astype(str)+"+"
dfM_2_g=dfM_2_g[~(dfM_2_g['ns']=="***+")].reset_index(drop=True)
dfM_2_g=dfM_2_g.groupby(by="transID").agg({'start':'min','ns':'sum','ends':'sum','end':'max','strand':'min','seqid':'min','nid':'count'})
dfM_2_g.rename({"nid":'count'},axis=1,inplace=True)
dfM_2_g.reset_index(inplace=True)
dfM_2_g.columns=['transID','start_min',"ns",'end','end_max','strand','seqid','count']
dfM_2_g["ns"]=dfM_2_g["ns"].str.rstrip("***+")
dfM_2_g["end"]=dfM_2_g["end"].str.rstrip("***+")
dfM_2_g["chain"]=dfM_2_g['seqid']+":"+dfM_2_g['strand']+'@'+dfM_2_g['end']+'-'+dfM_2_g['ns']
dfM_2_g=dfM_2_g[['transID','start_min','end_max','chain','count']]

del transcriptSet
dfM_2=dfM_2[gff3Cols+["transID"]]
dfM_2

,seqid,source,type,start,end,score,strand,phase,attributes,transID
0,CM000663.2,BestRefSeq,transcript,11874,14409,.,+,.,ID=CHS.1.1;Parent=CHS.1;gene_name=DDX11L1;Dbxr...,CHS.1.1
1,CM000663.2,BestRefSeq,exon,11874,12227,.,+,.,Parent=CHS.1.1,CHS.1.1
2,CM000663.2,BestRefSeq,exon,12613,12721,.,+,.,Parent=CHS.1.1,CHS.1.1
3,CM000663.2,BestRefSeq,exon,13221,14409,.,+,.,Parent=CHS.1.1,CHS.1.1
4,CM000663.2,Gnomon,transcript,184878,199860,.,-,.,ID=CHS.10.1;Parent=CHS.10;gene_name=LOC1027238...,CHS.10.1
5,CM000663.2,Gnomon,exon,184878,185350,.,-,.,Parent=CHS.10.1,CHS.10.1
6,CM000663.2,Gnomon,exon,185491,185559,.,-,.,Parent=CHS.10.1,CHS.10.1
7,CM000663.2,Gnomon,exon,186317,186469,.,-,.,Parent=CHS.10.1,CHS.10.1
8,CM000663.2,Gnomon,exon,187129,187287,.,-,.,Parent=CHS.10.1,CHS.10.1
9,CM000663.2,Gnomon,exon,187380,187577,.,-,.,Parent=CHS.10.1,CHS.10.1


In [76]:
# Step 1 - check if there are any that correspond to any intron chains entirely
#          these are the ones that already exist
intron_compatible=dfM_2_g.merge(dfM_g,how='inner',on='chain')
intron_compatible=intron_compatible[['transID_x','transID_y']]
intron_compatible.columns=['transID_prim','transID_alt']
intron_compatible.to_csv(outDir+"/intron_compatible.csv",index=False)

# step 2, check any single-exon transcripts that are contained within any other single-exon transcripts
singleDF_2=dfM_2[~(dfM_2["transID"].isin(dfM_2_g["transID"]))&(dfM_2['type']=='transcript')].reset_index(drop=True)
singleDF=dfM[~(dfM["transID"].isin(dfM_g["transID"]))&(dfM['type']=='transcript')].reset_index(drop=True)
singleDF=singleDF_2.merge(singleDF,on=['seqid','start','end','strand'],how="inner")
singleDF=singleDF[["transID_x","transID_y"]].reset_index(drop=True)
singleDF.columns=['transID_prim','transID_alt']
singleDF.to_csv(outDir+"/singleDF.csv",index=False)

# Step 2.5 we can now take the dataframes above, concatenate them and process
# these transcripts belong to genes that are guaranteed to exist on the primary scaffolds so we can move them
# and the rest of the sibling transcripts to the primary scaffolds
# and remove from the alternative scaffolds
perfectMatch=pd.concat([intron_compatible,singleDF])
mapDF=pd.read_csv(outDir+"/gmap_map.csv")
mapDF['uid']=mapDF['uid'].astype(str)
perfectMatch=perfectMatch.merge(mapDF,how='left',left_on='transID_alt',right_on='uid')
perfectMatch.drop('uid',axis=1,inplace=True)
perfectMatch.columns=['transID_prim','transID_alt','transID_alt_2']
perfectMatch.to_csv(outDir+"/perfectMatch_gmap.csv",index=False)
perfectMatch

,transID_prim,transID_alt,transID_alt_2
0,CHS.10151.4,1530,CHS.59603.1
1,CHS.10510.1,7327,CHS.10533.1
2,CHS.10610.1,201,CHS.13396.1
3,CHS.10610.2,202,CHS.13396.2
4,CHS.10994.1,217,CHS.13403.1
5,CHS.10994.1,265,CHS.13433.1
6,CHS.10997.10,222,CHS.13406.10
7,CHS.10997.10,275,CHS.13436.6
8,CHS.11001.11,236,CHS.13409.2
9,CHS.11001.11,293,CHS.13440.8


In [77]:
# now we can also add any other transcripts that belong to same alt genes

dfT=dfM[dfM["type"]=="transcript"].reset_index(drop=True)
dfT=dfT.merge(mapDF,how='left',left_on='transID',right_on='uid')
dfT['geneID_alt']="CHS."+dfT.QNAME.str.split(".",expand=True)[1]
perfectMatch["geneID_alt"]="CHS."+perfectMatch.transID_alt_2.str.split(".",expand=True)[1]
perfectMatch["geneID_prim"]="CHS."+perfectMatch.transID_prim.str.split(".",expand=True)[1]
dfT=dfT[dfT['geneID_alt'].isin(set(perfectMatch['geneID_alt']))].reset_index(drop=True)
dfT=dfT.merge(perfectMatch[["geneID_alt",'geneID_prim']],how='left',left_on='geneID_alt',right_on='geneID_alt')
dfT[["QNAME",'geneID_prim']].to_csv(outDir+"/additionalTrans_gmap.csv",index=False)
dfT

,seqid,source,type,start,end,score,strand,phase,attributes,transID,QNAME,uid,geneID_alt,geneID_prim
0,CM000670.2,ref,transcript,143790920,144172229,.,-,.,ID=100,100,CHS.54875.2,100,CHS.54875,CHS.54741
1,CM000668.2,ref,transcript,31971869,32010189,.,+,.,ID=10000,10000,CHS.49555.2,10000,CHS.49555,CHS.46019
2,CM000668.2,ref,transcript,31971869,32010225,.,+,.,ID=10001,10001,CHS.49555.3,10001,CHS.49555,CHS.46019
3,CM000668.2,ref,transcript,31980391,31981446,.,+,.,ID=10002,10002,CHS.49555.4,10002,CHS.49555,CHS.46019
4,CM000668.2,ref,transcript,32255173,32350039,.,+,.,ID=10028,10028,CHS.49573.1,10028,CHS.49573,CHS.46041
5,CM000668.2,ref,transcript,32255173,32500634,.,+,.,ID=10029,10029,CHS.49573.2,10029,CHS.49573,CHS.46041
6,CM000678.2,ref,transcript,15594369,16121482,.,-,.,ID=1003,1003,CHS.20838.1,1003,CHS.20838,CHS.19193
7,CM000678.2,ref,transcript,15594369,16121482,.,-,.,ID=1003,1003,CHS.20838.1,1003,CHS.20838,CHS.19193
8,CM000668.2,ref,transcript,32255173,32668073,.,+,.,ID=10030,10030,CHS.49573.3,10030,CHS.49573,CHS.46041
9,CM000668.2,ref,transcript,32718005,32718827,.,+,.,ID=10039,10039,CHS.49583.1,10039,CHS.49583,CHS.46062


In [44]:
# now we can see if any alts have been identified on multiple primary genes

,transID_prim,transID_alt,transID_alt_2
0,CHS.10151.4,1530,CHS.59603.1
1,CHS.10510.1,7327,CHS.10533.1
2,CHS.10610.1,201,CHS.13396.1
3,CHS.10610.2,202,CHS.13396.2
4,CHS.10994.1,217,CHS.13403.1
5,CHS.10994.1,265,CHS.13433.1
6,CHS.10997.10,222,CHS.13406.10
7,CHS.10997.10,275,CHS.13436.6
8,CHS.11001.11,236,CHS.13409.2
9,CHS.11001.11,293,CHS.13440.8


In [29]:
# Step 3 - find all that have at least one shared intron
#          these are the ones that need to be added as new transcripts to pre-existing genes

# to do this - create a dataframe with introns instead of exons
df=pd.read_csv(outDir+"/dfGFF_gmap.gtf",sep="\t",names=gff3Cols)

df=df[df["type"]=="exon"].reset_index(drop=True)
df["transID"]=df.attributes.str.split("Parent=",expand=True)[1].str.split(";",expand=True)[0]
df["start"]=df["start"].astype(int)
df["end"]=df["end"].astype(int)-1
# some pseudogenes are still present

dfI=df[["seqid","strand","type","start","end","transID"]].reset_index(drop=True)
dfI=dfI.sort_values(by=["transID","start"]).reset_index(drop=True)
dfI["ns"]=dfI.start.astype(str).shift(-1)
dfI["nid"]=dfI.transID.shift(-1)
dfI["ns"]=np.where(dfI["transID"]==dfI["nid"],dfI["ns"],"***")
dfI["ns"]=dfI.ns
dfI["ends"]=dfI.end
dfI["start"]=dfI.ends
dfI["end"]=dfI.ns
dfI=dfI[~(dfI["end"]=="***")].reset_index(drop=True)
dfI.drop(["ns","nid","ends"],axis=1,inplace=True)
dfI["end"]=dfI["end"].astype(int)
dfI["id"]=dfI["seqid"]+":"+dfI["strand"]+"@"+dfI["start"].astype(str)+"-"+dfI["end"].astype(str)

dfI.sort_values(by="id",ascending=True)

,seqid,strand,type,start,end,transID,id
3012,CM000663.2,+,exon,112956956,112960477,10368,CM000663.2:+@112956956-112960477
3010,CM000663.2,+,exon,112957722,112960477,10367,CM000663.2:+@112957722-112960477
3011,CM000663.2,+,exon,112960575,112966726,10367,CM000663.2:+@112960575-112966726
3013,CM000663.2,+,exon,112960575,112967492,10368,CM000663.2:+@112960575-112967492
3125,CM000663.2,+,exon,12857147,12858985,10381,CM000663.2:+@12857147-12858985
3126,CM000663.2,+,exon,12859296,12861530,10381,CM000663.2:+@12859296-12861530
3127,CM000663.2,+,exon,12862108,12865291,10381,CM000663.2:+@12862108-12865291
3128,CM000663.2,+,exon,12916687,12917668,10382,CM000663.2:+@12916687-12917668
3150,CM000663.2,+,exon,12916687,12917668,10391,CM000663.2:+@12916687-12917668
2947,CM000663.2,+,exon,12916687,12917668,10350,CM000663.2:+@12916687-12917668


In [10]:
df=pd.read_csv(outDir+"/chessPrim.gff",sep="\t",names=gff3Cols)

df=df[df["type"]=="exon"].reset_index(drop=True)
df["transID"]=df.attributes.str.split("Parent=",expand=True)[1].str.split(";",expand=True)[0]
df["start"]=df["start"].astype(int)
df["end"]=df["end"].astype(int)
# some pseudogenes are still present

dfI_2=df[["seqid","strand","type","start","end","transID"]].reset_index(drop=True)
dfI_2=dfI_2.sort_values(by=["transID","start"]).reset_index(drop=True)
dfI_2["ns"]=dfI_2.start.astype(str).shift(-1)
dfI_2["nid"]=dfI_2.transID.shift(-1)
dfI_2["ns"]=np.where(dfI_2["transID"]==dfI_2["nid"],dfI_2["ns"],"***")
dfI_2["ns"]=dfI_2.ns
dfI_2["ends"]=dfI_2.end
dfI_2["start"]=dfI_2.ends
dfI_2["end"]=dfI_2.ns
dfI_2=dfI_2[~(dfI_2["end"]=="***")].reset_index(drop=True)
dfI_2.drop(["ns","nid","ends"],axis=1,inplace=True)
dfI_2["end"]=dfI_2["end"].astype(int)
dfI_2["id"]=dfI_2["seqid"]+":"+dfI_2["strand"]+"@"+dfI_2["start"].astype(str)+"-"+dfI_2["end"].astype(str)

dfI_2.sort_values(by="id",ascending=True)

,seqid,strand,type,start,end,transID,id
715386,CM000663.2,+,exon,100007156,100011365,CHS.2329.6,CM000663.2:+@100007156-100011365
715398,CM000663.2,+,exon,100007156,100011365,CHS.2329.9,CM000663.2:+@100007156-100011365
715393,CM000663.2,+,exon,100007156,100011365,CHS.2329.7,CM000663.2:+@100007156-100011365
715367,CM000663.2,+,exon,100007156,100011365,CHS.2329.3,CM000663.2:+@100007156-100011365
715349,CM000663.2,+,exon,100007156,100011365,CHS.2329.15,CM000663.2:+@100007156-100011365
715343,CM000663.2,+,exon,100007156,100011365,CHS.2329.13,CM000663.2:+@100007156-100011365
715335,CM000663.2,+,exon,100007156,100011365,CHS.2329.12,CM000663.2:+@100007156-100011365
715379,CM000663.2,+,exon,100007156,100011365,CHS.2329.5,CM000663.2:+@100007156-100011365
715317,CM000663.2,+,exon,100007156,100011365,CHS.2329.1,CM000663.2:+@100007156-100011365
715323,CM000663.2,+,exon,100007156,100011365,CHS.2329.10,CM000663.2:+@100007156-100011365


In [56]:
dfI_2[(dfI_2["start"]-407669).abs()<10]

,seqid,strand,type,start,end,transID,id
2055983,CM000670.2,+,exon,407674,413073,CHS.52513.8,CM000670.2:+@407674-413073


In [20]:
# merge two dataframes on introns
dfI=dfI_2.merge(dfI,on="id",how="inner")
dfI

,seqid_x,strand_x,type_x,start_x,end_x,transID_x,id,seqid_y,strand_y,type_y,start_y,end_y,transID_y
0,CM000673.2,+,exon,118978856,118978957,CHS.10150.1,CM000673.2:+@118978856-118978957,CM000673.2,+,exon,118978856,118978957,7459
1,CM000673.2,+,exon,118978856,118978957,CHS.10150.2,CM000673.2:+@118978856-118978957,CM000673.2,+,exon,118978856,118978957,7459
2,CM000673.2,+,exon,118978856,118978957,CHS.10150.3,CM000673.2:+@118978856-118978957,CM000673.2,+,exon,118978856,118978957,7459
3,CM000673.2,+,exon,118978856,118978957,CHS.10150.4,CM000673.2:+@118978856-118978957,CM000673.2,+,exon,118978856,118978957,7459
4,CM000673.2,-,exon,118997591,118997728,CHS.10151.4,CM000673.2:-@118997591-118997728,CM000673.2,-,exon,118997591,118997728,1530
5,CM000673.2,+,exon,118998317,118998430,CHS.10152.1,CM000673.2:+@118998317-118998430,CM000673.2,+,exon,118998317,118998430,7460
6,CM000673.2,+,exon,118998317,118998430,CHS.10152.1,CM000673.2:+@118998317-118998430,CM000673.2,+,exon,118998317,118998430,7462
7,CM000673.2,+,exon,118998317,118998430,CHS.10152.1,CM000673.2:+@118998317-118998430,CM000673.2,+,exon,118998317,118998430,7463
8,CM000673.2,+,exon,118998317,118998430,CHS.10152.1,CM000673.2:+@118998317-118998430,CM000673.2,+,exon,118998317,118998430,7464
9,CM000673.2,+,exon,118998317,118998430,CHS.10152.10,CM000673.2:+@118998317-118998430,CM000673.2,+,exon,118998317,118998430,7460


In [ ]:
# extract geneID
dfI["geneID"]="CHS."+dfI["transID_x"].str.split(".",expand=True)[1]
# check if any have more than one associated geneID (should not be the case ever)
oneExonShared=dfI.groupby(by="transID_y").agg({"geneID":{"genes":lambda x: set(x),"geneCount":lambda x: len(set(x))}}).reset_index()
oneExonShared.columns=["transID_alt","geneID_prim","geneCount_prim"]
oneExonShared

# separate into those with
oneExonShared_single=oneExonShared[oneExonShared["geneCount_prim"]==1].reset_index(drop=True)
oneExonShared_single.to_csv(outDir+"/oneIntronShared_single_gmap.csv",index=False)
oneExonShared_multi=oneExonShared[oneExonShared["geneCount_prim"]>1].reset_index(drop=True)
oneExonShared_multi.to_csv(outDir+"/oneIntronShared_multi_gmap.csv",index=False)
display(oneExonShared_single)
display(oneExonShared_multi)

In [ ]:
# move transcripts

# not yet

In [ ]:
# step 4. see if there is significant overlap between any features
# perhaps with bedtools, maybe something else

In [ ]:
# for the remainder, we should first find those alignments that have good alignment quality, and figure out which one whould be used

In [ ]:
# first let's create geneIDs for all transcripts
# - need to identify overlapping transcripts
dfT=df[df["type"]=="transcript"].reset_index(drop=True)
dfT["start"]=dfT['start'].astype(int)
dfT["end"]=dfT['end'].astype(int)
dfT.sort_values(by=['seqid','strand','start','end'],ascending=True,inplace=True)
dfT.reset_index(drop=True,inplace=True)
dfT

dup_dfT=dfT[dfT.duplicated(["seqid","start","end","strand"],keep=False)].reset_index()
dup_dfT["transID"]=dup_dfT['attributes'].str.split("ID=",expand=True)[1].str.split(";",expand=True)[0]
dfT.groupby(by=["seqid","start","end","strand"]).agg({"type":'min',\
                                                       "source":'min',\
                                                       "score":'min',\
                                                       "phase":'min',\
                                                       "attributes":'min'}).reset_index()
dup_dfT

# now need to reassign transcript IDS to all exons that belong to duuplicated transcripts
dfE=df[df['type']=='exon'].reset_index(drop=True)
dfE_dupT

